## Geo-Visualisierung von Tweets (für Hamburger Stadtteile)

### 0. Imports  

In [1]:
import geopandas as gpd
import pandas as pd

from tweets_hh.Mapping.mapping import Map, make_polygons

### 1. Daten

**Geodaten**

Die Geodaten haben wir über eine Overpass Query von openstreetmap.org. Der entsprechende Code findet sich unter /tweets-hh/OSM/overpass_api.py. 
Overpass liefert die Daten in einem Format, welches für Visualisierungen mithilfe der Python-Bibliothek `Folium` unbrauchbar ist. Daher haben wir im selben Skript aus der HTTP-Response "händisch" eine GeoJSON-Datei erstellt (Hamburg.json). 
Außerdem wurden die abgefragten Stadtteilgrenzen als `Linestrings` und nicht als Polygone geliefert. Die eigens geschriebene Funktion `make_polygons` bedient sich der Funktion `linemerge` aus der `shapely` Bibliothek und wandelt die Geodaten in Polygone um. Mit Ausnahme von Volksdorf und Duvenstedt hat dieser Ansatz zu "lochfreien" Polygonen geführt. 

In [2]:
hamburg = gpd.read_file('tweets_hh/OSM/Hamburg.geojson')[["name", "geometry"]].to_crs(epsg = 4326)
hamburg["geometry"] = make_polygons(hamburg.geometry) 


**Tweets**

Enthält alle Daten, die die Twitter API zu einem Tweet bereitstellt und außerdem bereits die Zuordnung zu einem Stadtteil sowie die Stimmung des Tweets.

In [3]:
tweets = pd.read_csv("tweets_hh/Sentiment/df_sent.csv", parse_dates=["created_at"], low_memory=False)
tweets.head(5)



,Unnamed: 0,_id,contributors,coordinates,created_at,display_text_range,entities.hashtags,entities.media,entities.symbols,entities.urls,...,user.utc_offset,user.verified,user.withheld_in_countries,full_text,clean_text,district,is_hamburg,district_is_loc,sentiment,sentiment_score
0,10,1460940945386614791,NaN,NaN,2021-11-17 12:00:39+00:00,"[0,140]",[],NaN,[],"[{""url"":""https://t.co/zYJ97B1Nk8"",""expanded_ur...",...,NaN,False,[],Welle. Am Mittwochmorgen ist ein Autofahrer be...,welle am mittwochmorgen ist ein autofahrer bei...,Harburg,True,True,neutral,2.095035
1,32,1460945030257029125,NaN,NaN,2021-11-17 12:16:52+00:00,"[0,140]","[{""text"":""Hamburg"",""indices"":[15,23]},{""text"":...",NaN,[],"[{""url"":""https://t.co/hCZBEcsbks"",""expanded_ur...",...,NaN,True,[],🔴 Aktuell - In #Hamburg #Moorburg wurde bei So...,aktuell in hamburg moorburg wurde bei sondieru...,Moorburg,True,True,neutral,7.957128
2,35,1460945358540918789,NaN,NaN,2021-11-17 12:18:11+00:00,"[15,52]",[],NaN,[],[],...,NaN,False,[],@horstiamhandy Schullotse in Eppendorf mit der...,horstiamhandy schullotse in eppendorf mit der ...,Eppendorf,True,True,neutral,8.781985
3,40,1460946079877373958,NaN,NaN,2021-11-17 12:21:03+00:00,NaN,"[{""text"":""Hamburg"",""indices"":[6,14]},{""text"":""...",NaN,[],"[{""url"":""https://t.co/J8ZbQlXBXT"",""expanded_ur...",...,NaN,True,[],⚠️ In #Hamburg #Moorburg wurde eine 500 lbs. s...,in hamburg moorburg wurde eine fünf null null ...,Moorburg,True,True,neutral,8.271572
4,42,1460946372719521792,NaN,NaN,2021-11-17 12:22:12+00:00,"[11,85]",[],NaN,[],[],...,NaN,False,[],@einsskunk in eppendorf wirst du gemobbt wenn ...,einsskunk in eppendorf wirst du gemobbt wenn d...,Eppendorf,True,True,neutral,3.532805


**Einwohnerdaten**

Die Einwohnerdaten (Stand 31.12.2020) stammen vom Statistikamt Nord, wobei der Datensatz von [Statista](https://de.statista.com/statistik/daten/studie/1081610/umfrage/bevoelkerungsreichste-stadtteile-in-hamburg) zur Verfügung gestellt wurde. Fünf Stadtteile waren nicht aufgeführt. Die entsprechenden Zahlen haben wir Wikipedia entnommen und manuell hinzugefügt.  

In [4]:
pop = pd.read_excel("tweets_hh/Mapping/statista_einwohner_hh.xlsx", 
    sheet_name="Daten", header=None, usecols="B:C", skiprows=5, names = ["name", "population"])

### 2. Visualisierung

Über die sogenannte `LayerControl` (rechts oben am Rand der Grafik) sind verschiedene Choroplethenkarten auswählbar.
- "Tweets [Datum]": Marker, die einen Tweet über seinen Inhalt lokalisieren. Die genaue Position hat keine Bedeutung, lediglich die Zugehörigkeit zu einem bestimmten Stadtteil. Die Farbe der Marker stellen die Stimmung des Tweets dar: grün für eine positive, rot für eine negative und gelb für eine neutrale Stimmung. Klickt man auf einen Marker, wird Datum und Uhrzeit des Tweets angezeigt sowie der Tweet an sich. Das Datum (ggf. auch Uhrzeit) kann selbst gewählt werden. Defaultmäßig werden die Tweets der nächsten 24h angezeigt. Falls keine Uhrzeit angegeben ist, wird von Mitternacht ausgegangen. Das Stundenintervall kann aber auch selbst gesetzt werden.
- "Tweets per district": Choroplethenkarte, die die Anzahl von Tweets pro Stadtteil wiedergibt. 
- "Tweets per resident": Choroplethenkarte, die die Anzahl von Tweets pro Einwohner*in eines Stadtteils anzeigt.
- "Ratio of positive to negative Tweets": Choroplethenkarte, die die Anzahl positiver Tweets ins Verhältnis zur Anzahl negativer Tweets setzt. Diese Metrik wurde gewählt, da der überwiegende Anteil an Tweets neutral ist, wir uns aber insbesondere für subjekte Tweets interessieren. 

In [5]:
%%capture --no-display

mp = Map(hamburg, tweets, pop)
my_map = mp.draw_map(on = "2021-12-06", exclude="Neuwerk") # hours_interval = 24  
# districts can be excluded (e.g. Neuwerk because of tweets per district)
my_map

### 3. Analyse

\- Stand 18.12.21, 2594 gefilterte Tweets - 

In absoluten Zahlen haben wir zu Harburg die meisten Tweets verzeichnet (550), danach Wilhelmsburg (283) und Bergedorf (169). Setzt man die Anzahl in Beziehung zur Einwohnerzahl eines Stadtteils, schlägt Neuwerk alle Rekorde: 0.5 Tweets pro Einwohner*in. Allerdings wohnen auf Neuwerk nur 30 Menschen. Auf dem zweiten Platz liegt Moorburg mit 0.05 Tweets pro Einwohner\*in, danach folgt Steinwerder mit 0.03 Tweets.
Das beste Verhältnis zwischen positiven und negativen Tweets hat Jenfeld mit einem Verhältnis von 1:1. Darauf folgen Hamm (0.80), HafenCity (0.70) und Wilhelmsburg sowie Sternschanze (jeweils 0.66). 

Insgesamt ist die Datenlage noch zu dürftig, um statistische Aussagen zu treffen. Jenfeld hat z.B. jeweils nur einen positiven und einen negativen Tweet. Die automatische Stimmungsanalyse ist noch weit entfernt von menschlicher Intuition (zumindest für informelle und maximal unstrukturierte Texte wie Tweets es sind, dazu noch in nicht-englischer Sprache) -> siehe Beispiele unten. 
Mit noch größerer Datenlage (mehrere Monate) ließen sich sicherlich Trends feststellen, idealerweise dann noch mit einem BERT-Modell, dass einem Feintuning auf einem eigens gelabelten Twitter-Datensatz unterzogen wurde. Für Standorte, die ländlich sind, müsste die Analyseebene noch grober gefasst werden, und/oder über noch längere Zeiträume Daten gesammelt werden. Stimmungseffekte einzelner Projekte / Events ließen sich vermutlich nicht extrahieren. Allein für 47 Hamburger Stadtteile wurden über einen Zeitraum von einem Monat keine Tweets verzeichnet (die eindeutig Hamburg zuzuordnen sind & keine Servicetweets von Verkehrsbetrieben sind etc.)

**Positive Tweets**

Jenfeld:

In [6]:
for idx, tweet in enumerate(tweets[(tweets.district == "Jenfeld") & (tweets.sentiment == "positive")]["full_text"]):
    print(idx, tweet)

0 @janinaschreibt Ah ja cool ...wäre in Jenfeld wenn dir das nicht zu weit ist 🙃🙌🏽


Hamm:

In [7]:
for idx, tweet in enumerate(tweets[(tweets.district == "Hamm") & (tweets.sentiment == "positive")]["full_text"]):
    print(idx, tweet)

0 @BrittaFietzke @meyerzwoo @EinAugenschmaus Habe bis 2015 in Hamburg-Hamm gewöhnt, ca. 550.- warm für 58 qm inkl. Balkon, nur 300 m zu U-Bahn und Bus, war ein Glückstreffer. Ich wurde über SAGA fündig. Ich würde dort wieder wohnen, nicht schnieke, aber gepflegt und zentrumsnah. Wünsche viel Erfolg bei der Wohnungssuche!
1 Jetzt quer durch hamburg nach hamm fahren weil ich schlüssel wegbringen muss ich liebe es
2 @Thhhh53550658 @sebastiansozi Ich wohne in Hamm. Hier futtert das auch jede*r 😂
3 Ich fahre gleich zur Autobahnkapelle Hamm einfach weil ich es kann


HafenCity:

In [8]:
for idx, tweet in enumerate(tweets[(tweets.district == "HafenCity") & (tweets.sentiment == "positive")]["full_text"]):
    print(idx, tweet)
# neue Wohnprojekte ("in Hafencity zu leben wäre so ein Traum vor allem weil da noch riesen Bezirke mit modernen Apartments gebaut werden [...]")
# interessante Museen und Führungen, Kino
# einige Tweets vermutlich aber auch nur aufgrund von Emojis und verkannter Ironie als positiv bewertet: 
# "Ich bin echt froh, dass ich nicht von der Waldschlösschenbrücke nach Pieschen muss: Elberadweg 
# - an der Albertbrücke weg vom ERW - am Bellevue wieder runter - an der Hafencity wieder hoch - am Eselnest wieder runter. Was für ein Spaß!!"

0 Wow: Kürzlich war ich mit weiteren Xingfluencern zu Gast in der Hafencity von Hamburg: Wir durften das imposante #newwork Konzept von #XING erleben. 👍 🤩 🏆 Die Einladung war ein Dankeschön für unsere Impulse als App-Tester*innen für die neue XING App, die bald gelauncht wird. https://t.co/gsXofmB5QM
1 @4rbeitsamt Komm mich in der hafencity besuchen 😂
2 Neue Luxus-Bude für reiche Rentner in der HafenCity https://t.co/ZCwdwUZ6XQ https://t.co/ACk0cnvnjz
3 Einzigartige Tour durch Hafencity und Speicherstadt

Sehr angenehm und informativ. Die Tour hat uns sehr gefallen und wir würden es sofort wieder machen. Wenn wir mal wieder in Hamburg sind, würden wir gerne nochmal mit unserem Greeter Rainer eine Tour machen.

Du möchtest Hambur https://t.co/o96v6fxmg6
4 @DHLPaket ein kleiner Hinweis für den Fahrer: Moin liebes DHL Team, 
das befahren des Lola-Rogge-Platz in der Hafencity ist strengstens untersagt. Grundsätzlich halten sich alle Paketboten daran. Dieser Fahrer jedoch nicht und ist rück

Wilhelmsburg:

In [9]:
for idx, tweet in enumerate(tweets[(tweets.district == "Wilhelmsburg") & (tweets.sentiment == "positive")]["full_text"]):
    print(idx, tweet)

# lecker Döner
# Impfaktion
# "Perle im Süden"
# dörflicher Charakter - Möglichkeit für Wohnungstausch mit Personen, die in der Innenstadt wohnen und Ruhe suchen
# Online-Dating Gesuch
# "[...] An Heiligabend wird es in Wilhelmsburg schneien!"

0 Oh Wilhelmsburg, du Perle im Süden!⚓️  Hier lässt es sich nicht nur gut am schönen Veringkanal entlang spazieren, sondern dort gibt es auch einige Restaurants und Cafés zu entdecken! 🍰 Also ab nach Wilhelmsburg! 💙 https://t.co/rvhiYvw4z7
1 @MissJonesTravel @Patchworkzauber Wenn ich mir das so durchlese, wäre unsere Wohnung im Poble Sec/BCN genau der Trubel gewesen, der Dir gefallen hätte, aber leider haben wir die nicht mehr... Idee: Wohnungstausch mit jemanden aus der City. Wilhelmsburg ist ja auch wirklich schon Dorf.
2 @GalaidaG Heeeeeeey…. Aber, aber, aber… dem widerspreche ich! An Heiligabend wird es in Wilhelmsburg schneien! 👀😂
3 Jaschhha: Hmm. diese Art von "Online-Dating" ist noch für mich neu. Ich bin manchmal ein bisschen verrückt und mache Dinge, die andere nicht verstehen, aber das ist mir egal. Wenn du mehr über mich wissen willst, schreib mir. PLZ-Bereich: Wilhelmsburg… https://t.co/Eg7BSQSESi
4 CN Food

Habe gerade mal den "Filet der Straße" Döner für Zuhause probiert:

Sternschanze:

In [10]:
for idx, tweet in enumerate(tweets[(tweets.district == "Sternschanze") & (tweets.sentiment == "positive")]["full_text"]):
    print(idx, tweet)

0 sternschanze https://t.co/9P8p7G4UG9
1 Sitze gerade beim Türken auf der Sternschanze uns es ist so schön anzusehen, dass alle Gäste, egal ob jung oder alt, blau oder kariert, wie selbstverständlich beim Reinkommen Perso und Impfzertifikat zücken. Das macht Hoffnung und es gibt halt doch noch Menschen 😊 #hamburg https://t.co/mFQGbo9uBp
